In [1]:
import pandas as pd
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt    
from fastparquet import ParquetFile

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score

from utils import remove_highly_correlated_features, feature_drop

import warnings
warnings.filterwarnings("ignore")

/cephfs/projects/ppashin/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_173229/2248810513.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
file_path = "Data/train_ai_comp_final_dp.parquet"
pf = ParquetFile(file_path)
df = pf.to_pandas()

In [3]:
fature_to_drop = remove_highly_correlated_features(df, threshold=0.94)
df.drop(columns=fature_to_drop, inplace=True)
df = feature_drop(df)
df = df.fillna(0)

Optuna

In [6]:
import optuna
from catboost import CatBoostClassifier

X = df.drop(["target"], axis = 1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 1.0, 100.0),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-8, 1.0),
    }

    model = CatBoostClassifier(**param, loss_function='Logloss')
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
best_model = CatBoostClassifier(**best_params)
best_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

[I 2024-03-11 17:29:26,666] A new study created in memory with name: no-name-4f12ccd2-964b-4d7e-b14b-3f8da52dbf00


0:	learn: 0.6901089	test: 0.6901159	best: 0.6901159 (0)	total: 80.9ms	remaining: 30.6s
100:	learn: 0.4589726	test: 0.4594493	best: 0.4594493 (100)	total: 6.34s	remaining: 17.5s
200:	learn: 0.3284557	test: 0.3293326	best: 0.3293326 (200)	total: 12.5s	remaining: 11.1s
300:	learn: 0.2540422	test: 0.2552969	best: 0.2552969 (300)	total: 18.7s	remaining: 4.84s
378:	learn: 0.2182098	test: 0.2197252	best: 0.2197252 (378)	total: 23.8s	remaining: 0us

bestTest = 0.2197252273
bestIteration = 378



[I 2024-03-11 17:29:51,337] Trial 0 finished with value: 0.9641080415307487 and parameters: {'iterations': 379, 'learning_rate': 0.0018214514701896244, 'depth': 8, 'l2_leaf_reg': 0.13923648506539132, 'border_count': 107, 'bagging_temperature': 6.388009669759876, 'random_strength': 6.065009944146732e-07}. Best is trial 0 with value: 0.9641080415307487.


0:	learn: 0.5540710	test: 0.5544735	best: 0.5544735 (0)	total: 89.8ms	remaining: 1m 9s


[I 2024-03-11 17:29:57,114] Trial 1 finished with value: 0.9640406839679379 and parameters: {'iterations': 771, 'learning_rate': 0.0907337864350032, 'depth': 9, 'l2_leaf_reg': 2.9955721382375857e-06, 'border_count': 19, 'bagging_temperature': 3.515027877040358, 'random_strength': 0.004076859439917506}. Best is trial 0 with value: 0.9641080415307487.


...

0:	learn: 0.6439145	test: 0.6440336	best: 0.6440336 (0)	total: 124ms	remaining: 2m 3s
100:	learn: 0.1372416	test: 0.8546970	best: 0.1456733 (63)	total: 10.7s	remaining: 1m 34s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.145673307
bestIteration = 63

Shrink model to first 64 iterations.


[I 2024-03-11 17:43:43,779] Trial 22 finished with value: 0.9640791740038298 and parameters: {'iterations': 995, 'learning_rate': 0.03125742185973268, 'depth': 9, 'l2_leaf_reg': 3.80014646505508e-06, 'border_count': 201, 'bagging_temperature': 29.728127692927778, 'random_strength': 0.1311106348339184}. Best is trial 6 with value: 0.9641850216025326.


0:	learn: 0.5985717	test: 0.5988757	best: 0.5988757 (0)	total: 92.8ms	remaining: 1m 3s


[W 2024-03-11 17:43:49,673] Trial 23 failed with parameters: {'iterations': 690, 'learning_rate': 0.06005759286395733, 'depth': 8, 'l2_leaf_reg': 6.839404574591889e-06, 'border_count': 244, 'bagging_temperature': 14.32040651248944, 'random_strength': 0.003056473630504032} because of the following error: CatBoostError('/src/catboost/catboost/private/libs/algo/tensor_search_helpers.cpp:99: This should be unreachable').
Traceback (most recent call last):
  File "/cephfs/projects/ppashin/.local/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_152839/816748112.py", line 20, in objective
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)
  File "/cephfs/projects/ppashin/.local/lib/python3.11/site-packages/catboost/core.py", line 5201, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_we

CatBoostError: /src/catboost/catboost/private/libs/algo/tensor_search_helpers.cpp:99: This should be unreachable

In [5]:
y_pred_proba = best_model.predict_proba(X_test)
y_pred = best_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {classification_report(y_test, y_pred)}')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba.ravel()[1::2])

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'ROC AUC: {roc_auc}')

Accuracy:               precision    recall  f1-score   support

           0       0.96      1.00      0.98    100192
           1       0.65      0.01      0.01      3731

    accuracy                           0.96    103923
   macro avg       0.80      0.50      0.50    103923
weighted avg       0.95      0.96      0.95    103923

Precision: 0.6451612903225806
Recall: 0.005360493165371214
ROC AUC: 0.7381234061157389


Категориальные фичи 

In [15]:
# Artem cat features 
cat_features_tmp = ['feature5','feature6','feature7','feature9','feature74','feature85','feature96','feature106','feature117','feature127','feature138','feature150','feature163','feature174','feature215','feature293','feature343','feature344','feature345','feature346','feature347','feature348','feature349','feature350','feature352','feature356','feature360','feature363','feature364','feature374','feature379','feature381','feature383','feature389','feature393','feature395','feature396','feature397','feature398','feature399','feature400','feature401','feature402','feature403','feature404','feature408','feature409','feature410','feature411','feature412','feature413','feature415','feature416','feature417','feature419','feature421','feature422','feature425','feature427','feature428','feature430','feature431','feature432','feature436','feature437','feature438','feature439','feature441','feature442','feature443','feature444','feature445','feature446','feature447','feature448','feature449','feature450','feature452','feature453','feature454','feature459','feature460','feature463','feature465','feature466','feature472','feature474','feature475','feature476','feature477','feature478','feature479','feature480','feature481','feature483','feature485','feature486','feature488','feature490','feature491','feature492','feature493','feature494','feature497','feature498','feature499','feature500','feature501','feature502','feature504','feature505','feature508','feature509','feature510','feature512','feature513','feature514','feature515','feature516','feature517','feature519','feature520','feature521','feature524','feature525','feature526','feature527','feature528','feature534','feature535','feature538','feature539','feature541','feature542','feature543','feature544','feature547','feature548','feature549','feature550','feature552','feature553','feature554','feature557','feature558','feature559','feature560','feature577','feature580','feature581','feature594','feature596','feature597','feature598','feature599','feature600','feature601','feature602','feature604','feature606','feature608','feature609','feature610','feature611','feature612','feature613','feature616','feature617','feature619','feature620','feature621','feature622','feature631','feature634','feature639','feature644','feature647','feature650','feature654','feature660','feature664','feature666','feature668','feature672','feature673','feature682','feature692','feature695','feature698','feature705','feature712','feature740','feature741','feature743','feature744','feature746','feature747','feature750','feature751','feature753','feature754','feature757','feature758','feature760','feature763','feature766','feature767','feature769','feature770','feature771','feature773','feature776','feature777','feature786','feature787','feature788','feature789','feature790','feature791','feature792','feature794','feature795','feature796','feature797','feature800','feature803','feature804','feature805','feature811','feature812','feature813','feature814','feature815','feature835','feature838','feature842','feature843','feature845','feature851','feature853','feature855','feature857','feature861','feature863','feature864','feature866','feature868','feature870','feature873','feature875','feature877','feature879','feature881','feature884','feature887','feature927','feature928','feature929','feature930','feature931','feature932','feature937','feature991','feature992','feature993','feature994','feature995','feature996','feature997','feature998','feature999','feature1000','feature1003','feature1062','feature1063','feature1064','feature1065','feature1066','feature1068','feature1069','feature1070','feature1071','feature1072','feature1073','feature1074','feature1075','feature1076']
cat_features = [feature for feature in cat_features_tmp if feature in df.columns]

In [16]:
cat_features = [col for col in df.columns if df[col].nunique() < 5 and col != "target"]

190

In [17]:
# convert cats to string
for col in cat_features:
    df[col] = df[col].astype(str)

In [18]:
X = df.drop(["target"], axis = 1)
y = df['target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=8, loss_function='Logloss', cat_features=cat_features)
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)

0:	learn: 0.6779823	test: 0.6779615	best: 0.6779615 (0)	total: 793ms	remaining: 13m 11s
100:	learn: 0.1840523	test: 0.1828898	best: 0.1828898 (100)	total: 1m 49s	remaining: 16m 12s
200:	learn: 0.1466391	test: 0.1459123	best: 0.1459123 (200)	total: 4m 29s	remaining: 17m 52s
300:	learn: 0.1403589	test: 0.1404598	best: 0.1404598 (300)	total: 7m 31s	remaining: 17m 28s
400:	learn: 0.1382218	test: 0.1390445	best: 0.1390445 (400)	total: 10m 19s	remaining: 15m 25s
500:	learn: 0.1370145	test: 0.1384657	best: 0.1384657 (500)	total: 13m 15s	remaining: 13m 11s
600:	learn: 0.1361270	test: 0.1381147	best: 0.1381147 (600)	total: 16m 29s	remaining: 10m 56s
700:	learn: 0.1354130	test: 0.1378616	best: 0.1378616 (700)	total: 19m 45s	remaining: 8m 25s
800:	learn: 0.1348415	test: 0.1377058	best: 0.1377058 (800)	total: 22m 56s	remaining: 5m 41s
900:	learn: 0.1343423	test: 0.1375738	best: 0.1375738 (900)	total: 26m 1s	remaining: 2m 51s
999:	learn: 0.1339338	test: 0.1374693	best: 0.1374693 (999)	total: 29m 7s

In [23]:
feature_weights = {feature: weight for feature, weight in zip(X.columns, model.feature_importances_)}
dict(sorted(feature_weights.items(), key=lambda item: item[1], reverse=True))

{'feature1004': 10.25550868976953,
 'feature341': 5.448038390055345,
 'feature318': 5.088408990697241,
 'feature1000': 3.4621059234682305,
 'feature994': 2.8880895645245,
 'feature988': 2.589907079047815,
 'feature920': 2.2319737769760652,
 'feature210': 2.2288415732108433,
 'feature356': 2.068587209501762,
 'feature951': 2.014806758640155,
 'feature320': 1.9601388914492945,
 'feature208': 1.9003488756101612,
 'feature713': 1.8027410623113953,
 'feature936': 1.7247850117706516,
 'feature942': 1.6868011865573591,
 'feature922': 1.6207836059054634,
 'feature950': 1.6150634588144794,
 'feature357': 1.5422446849839793,
 'feature349': 1.4763411804980442,
 'feature941': 1.3130462302408705,
 'feature935': 1.1653183636486504,
 'feature444': 1.1559362876802721,
 'feature783': 1.1156465260760717,
 'feature940': 1.106815295659792,
 'feature1': 1.100492397898269,
 'feature551': 1.0996554580530191,
 'feature990': 1.077964538732958,
 'feature191': 1.0667436536315469,
 'feature534': 0.998438245418040

In [19]:
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {classification_report(y_test, y_pred)}')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba.ravel()[1::2])

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'ROC AUC: {roc_auc}')

Accuracy:               precision    recall  f1-score   support

           0       0.96      1.00      0.98     75146
           1       0.60      0.00      0.01      2797

    accuracy                           0.96     77943
   macro avg       0.78      0.50      0.49     77943
weighted avg       0.95      0.96      0.95     77943

Precision: 0.6
Recall: 0.0032177332856632105
ROC AUC: 0.7418843837886655


Отбор по feature importance

In [6]:
feature_weights = {feature: weight for feature, weight in zip(X.columns, model.feature_importances_)}
selected_features = [k for k, v in feature_weights.items() if v > 0]
f" отобралось {len(selected_features)} фич из {len(feature_weights)} "

(561, 749)

In [11]:
X_sel = df[selected_features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=8, loss_function='Logloss')
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

0:	learn: 0.6777413	test: 0.6777841	best: 0.6777841 (0)	total: 90.8ms	remaining: 1m 30s
100:	learn: 0.1816663	test: 0.1828781	best: 0.1828781 (100)	total: 7.47s	remaining: 1m 6s
200:	learn: 0.1452007	test: 0.1475164	best: 0.1475164 (200)	total: 15.5s	remaining: 1m 1s
300:	learn: 0.1393600	test: 0.1426631	best: 0.1426631 (300)	total: 23.9s	remaining: 55.6s
400:	learn: 0.1374007	test: 0.1415068	best: 0.1415068 (400)	total: 32.1s	remaining: 48s
500:	learn: 0.1361880	test: 0.1409738	best: 0.1409738 (500)	total: 39.8s	remaining: 39.7s
600:	learn: 0.1353036	test: 0.1406660	best: 0.1406660 (600)	total: 47.4s	remaining: 31.5s
700:	learn: 0.1345623	test: 0.1404493	best: 0.1404493 (700)	total: 54.8s	remaining: 23.4s
800:	learn: 0.1338575	test: 0.1402639	best: 0.1402639 (800)	total: 1m 2s	remaining: 15.5s
900:	learn: 0.1332664	test: 0.1401084	best: 0.1401084 (900)	total: 1m 9s	remaining: 7.63s
999:	learn: 0.1327011	test: 0.1399924	best: 0.1399924 (999)	total: 1m 16s	remaining: 0us

bestTest = 0.1

In [12]:
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {classification_report(y_test, y_pred)}')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba.ravel()[1::2])

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'ROC AUC: {roc_auc}')

Accuracy:               precision    recall  f1-score   support

           0       0.96      1.00      0.98    100192
           1       0.77      0.00      0.01      3731

    accuracy                           0.96    103923
   macro avg       0.87      0.50      0.49    103923
weighted avg       0.96      0.96      0.95    103923

Precision: 0.7692307692307693
Recall: 0.002680246582685607
ROC AUC: 0.7406038834812663


In [9]:
feature_weights = {feature: weight for feature, weight in zip(X.columns, model.feature_importances_)}
selected_features = [k for k, v in dict(sorted(feature_weights.items(), key=lambda item: item[1], reverse=True)).items() if v > 0]
len(selected_features), len(feature_weights)

(561, 749)

In [10]:
feature_weights

{'id': 0.03807066600838285,
 'sample_ml_new': 0.0,
 'feature1': 1.2488766221397836,
 'feature2': 0.1598991122892265,
 'feature3': 0.08070800397400199,
 'feature4': 0.1116965235891375,
 'feature5': 0.06052095579327117,
 'feature6': 0.18210495553714393,
 'feature7': 0.05118671231306596,
 'feature9': 0.09000518155062921,
 'feature12': 0.07257652168078543,
 'feature18': 0.13056974251327808,
 'feature19': 0.0552117134006944,
 'feature22': 0.016558461033134034,
 'feature23': 2.685235435876654e-05,
 'feature24': 0.0056580833675225805,
 'feature25': 0.07387581848902283,
 'feature26': 0.0,
 'feature27': 3.734147911872943e-05,
 'feature28': 0.00016417105350564678,
 'feature30': 0.016597221193952943,
 'feature31': 0.010995943441797522,
 'feature33': 0.0,
 'feature35': 0.45975151153987803,
 'feature36': 0.008317434195305954,
 'feature37': 0.09306898550918699,
 'feature38': 0.0327386469618588,
 'feature41': 0.035665957179934295,
 'feature42': 0.02063912858843175,
 'feature43': 0.242487277290389,
 '

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt    
from fastparquet import ParquetFile

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, roc_auc_score

from utils import remove_highly_correlated_features, feature_drop

import warnings
warnings.filterwarnings("ignore")


file_path = "Data/train_ai_comp_final_dp.parquet"
pf = ParquetFile(file_path)
df = pf.to_pandas()

fature_to_drop = remove_highly_correlated_features(df, threshold=0.94)
df.drop(columns=fature_to_drop, inplace=True)
df = feature_drop(df)
df = df.fillna(0)

# Artem cat features 
cat_features_tmp = ['feature5','feature6','feature7','feature9','feature74','feature85','feature96','feature106','feature117','feature127','feature138','feature150','feature163','feature174','feature215','feature293','feature343','feature344','feature345','feature346','feature347','feature348','feature349','feature350','feature352','feature356','feature360','feature363','feature364','feature374','feature379','feature381','feature383','feature389','feature393','feature395','feature396','feature397','feature398','feature399','feature400','feature401','feature402','feature403','feature404','feature408','feature409','feature410','feature411','feature412','feature413','feature415','feature416','feature417','feature419','feature421','feature422','feature425','feature427','feature428','feature430','feature431','feature432','feature436','feature437','feature438','feature439','feature441','feature442','feature443','feature444','feature445','feature446','feature447','feature448','feature449','feature450','feature452','feature453','feature454','feature459','feature460','feature463','feature465','feature466','feature472','feature474','feature475','feature476','feature477','feature478','feature479','feature480','feature481','feature483','feature485','feature486','feature488','feature490','feature491','feature492','feature493','feature494','feature497','feature498','feature499','feature500','feature501','feature502','feature504','feature505','feature508','feature509','feature510','feature512','feature513','feature514','feature515','feature516','feature517','feature519','feature520','feature521','feature524','feature525','feature526','feature527','feature528','feature534','feature535','feature538','feature539','feature541','feature542','feature543','feature544','feature547','feature548','feature549','feature550','feature552','feature553','feature554','feature557','feature558','feature559','feature560','feature577','feature580','feature581','feature594','feature596','feature597','feature598','feature599','feature600','feature601','feature602','feature604','feature606','feature608','feature609','feature610','feature611','feature612','feature613','feature616','feature617','feature619','feature620','feature621','feature622','feature631','feature634','feature639','feature644','feature647','feature650','feature654','feature660','feature664','feature666','feature668','feature672','feature673','feature682','feature692','feature695','feature698','feature705','feature712','feature740','feature741','feature743','feature744','feature746','feature747','feature750','feature751','feature753','feature754','feature757','feature758','feature760','feature763','feature766','feature767','feature769','feature770','feature771','feature773','feature776','feature777','feature786','feature787','feature788','feature789','feature790','feature791','feature792','feature794','feature795','feature796','feature797','feature800','feature803','feature804','feature805','feature811','feature812','feature813','feature814','feature815','feature835','feature838','feature842','feature843','feature845','feature851','feature853','feature855','feature857','feature861','feature863','feature864','feature866','feature868','feature870','feature873','feature875','feature877','feature879','feature881','feature884','feature887','feature927','feature928','feature929','feature930','feature931','feature932','feature937','feature991','feature992','feature993','feature994','feature995','feature996','feature997','feature998','feature999','feature1000','feature1003','feature1062','feature1063','feature1064','feature1065','feature1066','feature1068','feature1069','feature1070','feature1071','feature1072','feature1073','feature1074','feature1075','feature1076']
cat_features = [feature for feature in cat_features_tmp if feature in df.columns]

# convert cats to string
for col in cat_features:
    df[col] = df[col].astype(str)


X = df.drop(["target"], axis = 1)
y = df['target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = CatBoostClassifier(iterations=1000, learning_rate=0.01, depth=8, loss_function='Logloss', cat_features=cat_features)
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)


# metrics
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {classification_report(y_test, y_pred)}')

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba.ravel()[1::2])

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'ROC AUC: {roc_auc}')


# sorted feature importance
feature_weights = {feature: weight for feature, weight in zip(X.columns, model.feature_importances_)}
dict(sorted(feature_weights.items(), key=lambda item: item[1], reverse=True))